In [12]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
import warnings
warnings.filterwarnings('ignore')


import re
import os, sys

import numpy as np # linear algebra
import spacy
from scipy.special import softmax


import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer

from config import SEED, PARTIAL_TRAIN, TEST_SIZE, NUM_LABELS 
from config import MAX_SEQUENCE_LENGTH, NUM_EPOCH, LEARNING_RATE, BATCH_SIZE
from config import ACCUMULATION_STEPS, INPUT_DIR, WORK_DIR, TOXICITY_COLUMN, DATA_DIR
from config import BERT_MODEL_NAME, FINE_TUNED_MODEL_PATH

from utils import set_seed, convert_lines_onfly, preprocess

In [2]:
device = torch.device('cuda')

set_seed(SEED)

## instantiate bert pretrained model and tokenizer
toxic_model = BertForSequenceClassification.from_pretrained(BERT_MODEL_NAME, num_labels=NUM_LABELS)
toxic_tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

## load saved model
toxic_model.load_state_dict(torch.load(FINE_TUNED_MODEL_PATH)) 
for p in toxic_model.parameters():
    p.requires_grad = False

## use gpu
toxic_model.cuda()

def predict_toxicity(sentence: str) -> float :
    """
    predict the toxicity level from a sentence
    """    

    toxic_model.eval()
    
    X = np.array([str(sentence)])
    test_preds = torch.zeros((len(X)))
    
    Xp = convert_lines_onfly(X, MAX_SEQUENCE_LENGTH, toxic_tokenizer)
    y_pred = toxic_model(torch.from_numpy(Xp).to(device)).logits
    test_preds[0] = test_preds[0] + torch.sigmoid(y_pred[0,0].cpu())

    return float(test_preds[0])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [16]:
# Preprocess text (username and link placeholders)
def preprocess_sentiment(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task = 'sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
sentiment_tokenizer = AutoTokenizer.from_pretrained(MODEL)

# PT
sentiment_model = AutoModelForSequenceClassification.from_pretrained(MODEL)

# label mapping
NEG = 'negative'
NET = 'neutral'
POS = 'positive'
labels = [NEG, NET, POS]

label2id = {k:v for k, v in zip(labels, range(3))}
id2label = {k:v for k, v in zip(range(3), labels)}

def predict_sentiment(sentence: str) :
    text = preprocess_sentiment(sentence)
    encoded_input = sentiment_tokenizer(text, return_tensors='pt')
    output = sentiment_model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    sentiment = {id2label[idx]:s for idx, s in enumerate(scores)}

    return sentiment

In [17]:
nlp = spacy.load("en_core_web_lg")

def split_text(sentence): 
    doc = nlp(sentence)
    sentences = [sent.string.strip() for sent in doc.sents]
    return sentences

In [18]:
sentence = "Oh sh*t!! What an awesome goal, I nearly missed it…"
# sentence = "Yet call out all Muslims for the acts of a few will get you pilloried.   So why is it okay to smear an entire religion over these few idiots?  Or is this because it's okay to bash Christian sects?"
# sentence = "Sorry to have to do this, but just to see if profanity filtering is enabled"

In [19]:
print("Sentence: ", sentence)
print("Toxicity: ", predict_toxicity(sentence))
print("Sentiment: ", predict_sentiment(sentence))
print("Splitted Sentence: ", split_text(sentence))

Sentence:  Oh sh*t!! What an awesome goal, I nearly missed it…
Toxicity:  0.673354983329773
Sentiment:  {'negative': 0.21319842, 'neutral': 0.23147435, 'positive': 0.5553271}
Splitted Sentence:  ['Oh sh*t!!', 'What an awesome goal, I nearly missed it…']


In [27]:
def handle_edge_case(sentence: str, toxic_threshold=0.3) -> float :
    sentences = split_text(sentence)
    for i in range(len(sentences)) :
        truncated = " ".join(sentences[:i] + sentences[i+1:])
        toxicity = predict_toxicity(truncated)
        sentiment = predict_sentiment(truncated)
        if toxicity < toxic_threshold and sentiment[POS] > sentiment[NEG] :
            return toxicity
    return predict_toxicity(sentence)

def predict_with_combined_toxicity_sentiment(sentence: str) -> float :
    toxicity = predict_toxicity(sentence)
    sentiment = predict_sentiment(sentence)
    
    score = toxicity
    
    if toxicity > 0.9 :
        if sentiment[NEG] > sentiment[POS] :
            score = toxicity
        else :
            score = handle_edge_case(sentence, toxic_threshold=0.1)
    elif toxicity > 0.3 :
        if sentiment[NEG] > sentiment[POS] :
            score = toxicity
        else :
            score = handle_edge_case(sentence, toxic_threshold=0.5)
    else : # toxicity < 0.3
        score = toxicity
            
    return score

In [28]:
predict_with_combined_toxicity_sentiment(sentence)

0.0033970933873206377